# First Task

#### 1. Install BeautifulSoup4 library

We need to install beautifulsoup4 library in the first place for web scrapping in this task.

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Siahaan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py37_0         153 KB  anaconda
    certifi-2019.11.28         |           py37_0         157 KB  anaconda
    conda-4.8.0                |           py37_1         3.1 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following packages will be UPDATED:

  beautifulsoup4     pkgs/main::beautifulsoup4-4.7.1-py37_1 --> anaconda::beautifulsoup4-4.8.1-py37_0
  openssl            conda-forge::openssl-1.1.1d-hfa6e2cd_0 --> anaconda::openssl-1.1.1-he774522_0

The following packages will be SUPERSEDED by a higher-priority channel:


#### 2. Import all needed libraries
All necessary library imported in this project.

In [15]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

#### 3. Web Scrapping the wikipedia page containing list of neighborhoods in Toronto
The following lines of code are for web scrapping the wikipedia page that contains list of neighborhoods in the city of Toronto.

In [16]:
# URL of the wikipedia page
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# Create beautiful soup object
soup = BeautifulSoup(source, 'lxml')

# Find table that contains the neighborhoods data in the page
table = soup.find('table').tbody

# Get all columns name
columns = [item.text.replace("\n", "") for item in table.find_all("th")]

# Get all neighborhood data
neighborhoods_data = []
for i,row in enumerate(table.find_all('tr')):
    neighborhoods_data.append([])
    for j, item in enumerate(row.find_all('td')):
        neighborhoods_data[i].append(item.text.replace("\n", ""))

neighborhoods_data = neighborhoods_data[1:]
neighborhoods_data = list(map(list, zip(*neighborhoods_data)))

# Create Dataframe
tor_neigh_df = pd.DataFrame({columns[i]: neighborhoods_data[i] for i in range(3)})
tor_neigh_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 4. Rename PostalCode Column
We want to rename one column in our dataframe.

In [17]:
tor_neigh_df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 5. Drop Not Assigned value in Borough Column
All rows with not assigned borough value will be dropped.

In [18]:
tor_neigh_df = tor_neigh_df[tor_neigh_df["Borough"]!="Not assigned"]
tor_neigh_df.reset_index(drop=True, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 6. Cell with Not Assigned neighborhood will be filled with the same value as the borough 

In [19]:
# Function to check neighborhood value and process it if it is still not assigned
def replace_neighborhood_na(row):
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
    return row

In [20]:
tor_neigh_df = tor_neigh_df.apply(replace_neighborhood_na, axis=1)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 7. Combine rows with same PostalCode

In [21]:
neigh_df = tor_neigh_df.groupby(by="PostalCode").agg({"Neighborhood": ", ".join})
neigh_df.head()

,Neighborhood
PostalCode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae


shape of the Dataframe

In [22]:
neigh_df.shape

(103, 1)